In [2]:
import paramiko
import subprocess

ip = "172.16.16.122"
user = "root"
pw = "asd123!@"
cmd = ""

def cc(cmd):
    cli = paramiko.SSHClient()                                      # SSH 접속 객체 생성
    cli.set_missing_host_key_policy(paramiko.AutoAddPolicy())       # 키 교환 알고리즘 해결
    cli.connect(ip, username=user, password=pw, port=22)            # 커넥터
    (stdin, stdout, stderr) = cli.exec_command(cmd)
    return stdout

def pp(stdout):
    for i in stdout:
        print(i)

def smb_install():
    cmd = "dnf -y install samba samba-client cifs-utils"
    stdout = cc(cmd)
    #pp(stdout)

def install_test():
    cmd = "rpm -qa | grep samba"
    stdout = cc(cmd)
    for i in stdout:
        if "samba-4.21.3-14" in i:
            print("삼바 서버 설치 완료")
        elif "samba-client-4.21.3-14" in i:
            print("삼바 클라이언트 설치 완료")
    #samba-4.21.3-14.el9_6.x86_64
    #samba-client-4.21.3-14.el9_6.x86_64

def useradd():
    cc("useradd -s/sbin/nologin smb")
    PASSWORD = "asd123!@"

    cmd = f'''
    expect -c "
    set timeout 10
    spawn smbpasswd -a smb
    expect \\"New SMB password:\\"
    send \\"{PASSWORD}\\r\\"
    expect \\"Retype new SMB password:\\"
    send \\"{PASSWORD}\\r\\"
    expect eof
    "
    '''
    cc(cmd)

def smb_account_test():
    cmd = "pdbedit --list"
    print(cc(cmd).read())

def make_directory():
    cmd = "mkdir -p /share/smb"
    cc(cmd)
    print(cc("ls /share").read())

def smb_set():
    f = open("/etc/samba/smb.conf", "r")
    data = f.read()
    if "[share]" not in data:
        cmd = """cd /etc/samba
                cp -ap smb.conf smb.conf.bak"""
        cc(cmd)
        cmd = """echo "[share]
            comment = Kong\'s SMB
            hosts allow = 172.16.0.0/16
            path = /share/smb
            read only = No
            valid users = smb
            write list = smb
        " >> /etc/samba/smb.conf"""
        cc(cmd)
    else:
        print("이미 설정이 있어서 설정파일 수정 안했어요!!!")

def permit():
    cmd = """chmod 777 /share
    chown smb:smb /share/smb"""
    cc(cmd)

def service_start():
    cmd = """systemctl restart smb
    systemctl status smb"""
    stdout = cc(cmd)
    for i in stdout:
        if "Active: active (running)" in i:
            print("smbd 실행 성공")
    cmd = """systemctl restart nmb
    systemctl status nmb"""
    stdout = cc(cmd)
    for i in stdout:
        if "Active: active (running)" in i:
            print("nmbd 실행 성공")

def go():
    smb_install()       # 설치
    install_test()      # 설치 확인
    useradd()           # smb 계정 생성 및 SMB 계정 등록
    smb_account_test()  # samba 등록 계정 확인
    make_directory()    # smb 계정으로 공유할 디렉터리 생성
    smb_set()           # /etc/samba/smb.conf 파일에 공유폴더 "[share]" 추가
    permit()            # /share : 모두 접근 가능 /share/smb : smb 계정만 접근 및 쓰기 가능
    service_start()     # smbd/nmbd 서비스 활성화

#실행부
go()

삼바 서버 설치 완료
삼바 클라이언트 설치 완료
b'smb:1002:\n'
b'smb\n'
이미 설정이 있어서 설정파일 수정 안했어요!!!
smbd 실행 성공
nmbd 실행 성공
